In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

trn = "C:/Users/satra/Downloads/jigsaw-agile-community-rules/train.csv"
tst = "C:/Users/satra/Downloads/jigsaw-agile-community-rules/test.csv"
df_trn = pd.read_csv(trn)
# df_trn = df_trn.sample(frac=.01, random_state=42).reset_index(drop=True)
df_tst = pd.read_csv(tst)

Index(['row_id', 'body', 'rule', 'subreddit', 'positive_example_1',
       'positive_example_2', 'negative_example_1', 'negative_example_2',
       'rule_violation'],
      dtype='object')

In [4]:
trn_rows = []
for idx, row in df_trn.iterrows():
  trn_rows.append({
    'body': row['positive_example_1'],
    'rule': row['rule'],
    'subreddit': row['subreddit'],
    'label': 1
  })

  trn_rows.append({
    'body': row['positive_example_2'],
    'rule': row['rule'],
    'subreddit': row['subreddit'],
    'label': 1
  })

  trn_rows.append({
    'body': row['negative_example_1'],
    'rule': row['rule'],
    'subreddit': row['subreddit'],
    'label': 0
  })

  trn_rows.append({
    'body': row['negative_example_2'],
    'rule': row['rule'],
    'subreddit': row['subreddit'],
    'label': 0
  })

trn_df = pd.DataFrame(trn_rows)

val_rows = []
for idx, row in df_trn.iterrows():
  val_rows.append({
    'body': row['body'],
    'rule': row['rule'],
    'subreddit': row['subreddit'],
    'label': row['rule_violation']
  })

val_df = pd.DataFrame(val_rows)

print (f'Train shape: {trn_df.shape}, Val shape: {val_df.shape}')

Train shape: (80, 4), Val shape: (20, 4)


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

body_vec = TfidfVectorizer(max_features=10000)
rule_vec = TfidfVectorizer()
sred_vec = TfidfVectorizer()

X_body_trn = body_vec.fit_transform(trn_df['body'])
X_rule_trn = rule_vec.fit_transform(trn_df['rule'])
X_sred_trn = sred_vec.fit_transform(trn_df['subreddit'])

X_body_val = body_vec.transform(val_df['body'])
X_rule_val = rule_vec.transform(val_df['rule'])
X_sred_val = sred_vec.transform(val_df['subreddit'])

Y_trn = trn_df['label']
Y_val = val_df['label']

In [7]:
from scipy.sparse import hstack, csr_matrix

X_trn = hstack([X_body_trn, X_rule_trn, X_sred_trn]).tocsr()
X_val = hstack([X_body_val, X_rule_val, X_sred_val]).tocsr()

print(f'X_trn shape: {X_trn.shape}, X_val shape: {X_val.shape}')

X_trn shape: (80, 1016), X_val shape: (20, 1016)


In [10]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

model = lgb.LGBMClassifier(objective='binary', random_state=42)
model.fit(X_trn, Y_trn)

probs = model.predict_proba(X_val)[:, 1]
auc = roc_auc_score(Y_val, probs)

print(f'AUC: {auc}')

[LightGBM] [Info] Number of positive: 40, number of negative: 40
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000100 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 123
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

C:\Users\satra\PycharmProjects\Kaggle_Contest\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
